### Required libraries in Colab

In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install jsonlines
!pip install lamini
!pip install torch-xla
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extension

In [ ]:
import datasets
import logging
import random
import torch
import transformers
import pandas as pd
#import jsonlines
import config
import torch_xla.core.xla_model as xm
import os
from tqdm import tqdm

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import Trainer

#evaluation
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

from myLLM_utilities import *


logger = logging.getLogger(__name__)
global_config = None

### Set up the model, test and training datasets, and tokenizer

In [ ]:
# finding available processor
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    # Check if TPU is available
    if 'COLAB_TPU_ADDR' in os.environ:
        device = xm.xla_device()
        print("Using TPU")
    else:
        # If neither GPU nor TPU is available, fall back to CPU
        device = torch.device("cpu")
        print("Using CPU")

Using CPU


In [ ]:
#dataset_path = "lamini/lamini_docs"
dataset_path = 'lamini/taylor_swift'
use_hf = True

In [ ]:
model_name = "EleutherAI/pythia-410m"

In [ ]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

DEBUG:myLLM_utilities:Config: datasets.path: lamini/taylor_swift
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-410m
verbose: true



...

Generating train split:   0%|          | 0/783 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___taylor_swift/default/0.0.0/8b30efae27d3168b883a2812e660f7728d62bee3.incomplete/taylor_swift-train-00000-00000-of-NNNNN.arrow


Generating test split:   0%|          | 0/87 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___taylor_swift/default/0.0.0/8b30efae27d3168b883a2812e660f7728d62bee3.incomplete/taylor_swift-test-00000-00000-of-NNNNN.arrow
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___taylor_swift/default/0.0.0/8b30efae27d3168b883a2812e660f7728d62bee3.incomplete/dataset_info.json


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 783
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 87
})


### Load the base model and assign appropriate processor to it

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-037436b4d830>", line 1, in <cell line: 1>
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 566, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3037, in from_pretrained
    resolved_archive_file = cached_file(pretrained_model_name_or_path, filename, **cached_file_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py", line 430, in cached_file
    resolved_file = hf_hub_download(
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py", line 118, in _inner_fn
    return fn(*args, **kwargs)
  File "/usr/local/

TypeError: object of type 'NoneType' has no len()

In [ ]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

### Define function to carry out inference

In [ ]:
def post_process(text):
    # Split the generated text into sentences
    sentences = text.split('.')

    # Remove repetitive sentences
    unique_sentences = list(set(sentences))

    # If the last sentence is not complete, remove it
    last_sentence = unique_sentences[-1].strip()
    if not last_sentence.endswith('.'):
        unique_sentences[-1] = ''

    # Reconstruct the text
    post_processed_text = '. '.join(unique_sentences)

    return post_processed_text

In [ ]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=300):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens,
    top_k=50,  # Adjust this value
    top_p=0.95
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]
  post_processed_answer = post_process(generated_text_answer)

  return post_processed_answer

### Try the base model

In [ ]:
test_text = test_dataset[0]['question']
print("Question input (test):", str(test_text + " answer with 3 sentences."))
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

## Finetuning base-model

### Setup training

In [ ]:
max_steps = 2000
trained_model_name = f"pythia410m-taylorswift-{max_steps}-steps"
output_dir = trained_model_name

In [ ]:
#checkpoint_path = checkpoint_path = os.path.join(output_dir, "checkpoint-240")
training_args = TrainingArguments(

  #resume_from_checkpoint=checkpoint_path,
  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=2,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Configure TPU if TPU is used
  #tpu_num_cores= xm.xrt_world_size() if 'COLAB_TPU_ADDR' in os.environ else None,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [ ]:
trainer =   Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

### Training...

In [ ]:
training_output = trainer.train()

Step,Training Loss,Validation Loss
120,1.824100,1.725532
240,1.003200,1.526828
360,1.141000,1.465355
480,0.606200,1.459430
600,0.441200,1.470840
720,0.408500,1.515570
840,0.279000,1.541076
960,0.254900,1.501477
1080,0.297200,1.547585
1200,0.238900,1.573092


INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gather'...
INFO:root:Joining rendezvous 'nested_gat

### Save model locally

In [ ]:
#if you want to save the trained model locally, on google colab use this code. but when the runtime disconnects, all data is removed
save_dir = f'{output_dir}/final'
trainer.save_model(save_dir)
print("Saved model to:", save_dir)

INFO:root:Joining rendezvous 'saving_checkpoint'...


Saved model to: pythia410m-taylorswift-2000steps/final


In [ ]:
#load the finetuned model that is saved locally
finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

### Save this model to huggingface hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
trainer.push_to_hub("kh-jalali/pythia410m-taylorswift-2000steps")

INFO:root:Joining rendezvous 'saving_checkpoint'...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/kh-jalali/pythia410m-taylorswift-2000steps/commit/45a81e1c02ed8d70e0811cbbfedb3028674dc81c', commit_message='kh-jalali/pythia410m-taylorswift-2000-steps', commit_description='', oid='45a81e1c02ed8d70e0811cbbfedb3028674dc81c', pr_url=None, pr_revision=None, pr_num=None)

## Using finetuned model

In [ ]:
#load the model from huggingface hub
finetuned_model = AutoModelForCausalLM.from_pretrained("kh-jalali/pythia410m-taylorswift-2000steps")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
finetuned_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

### Run the finetuned model

In [ ]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_model, tokenizer))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Has Taylor Swift written songs for other artists?
Finetuned slightly model's answer: 
 Stay tuned!

9.  What is the release date of her second album, Fearless?The Fearless album was released in October 2010.  Stay tuned!

6.  Taylor Swift's fourth album, Fearless, was released in April 2012.  Taylor Swift's second album, Fearless, was released in November 2010.  Taylor Swift's sixth album, Fearless, was released in April 2014.  Taylor Swift's third album, Fearless, was released in April 2011.  What is the release date of her fifth album, Fearless?The Fearless album was released in March 2013.  Stay tuned!

10.  Taylor Swift's seventh album, Fearless, was.  What is the release date of her fourth album, Fearless?The Fearless album was released in March 2012.  Stay tuned!

5.  Stay tuned!

7.  Stay tuned!

8.  What is the release date of her seventh album, Fearless?The Fearless album was released in March 2015.  What is the release date of her sixth album, Fearless?

In [ ]:
print(test_dataset[0]['answer'])

Yes, Taylor Swift has written songs for other artists. Some notable examples include This Is What You Came For by Calvin Harris featuring Rihanna, Better Man by Little Big Town, and You'll Always Find Your Way Back Home by Miley Cyrus.


### Evaluating

In [ ]:
def bleu_similarity(predicted_answer , base_answer):
  gen_tokens = predicted_answer.split()
  ref_tokens = base_answer.split()
  # Calculate BLEU score 1-gram
  bleu_score = sentence_bleu([ref_tokens], gen_tokens , weights = (1,0,0,0))
  print(f"BLEU Score: {bleu_score}")
  return bleu_score

In [ ]:
def rouge_similarity(predicted_answer , base_answer):
  gen_tokens = predicted_answer.split()
  ref_tokens = base_answer.split()

  # Prepare input for ROUGE
  hyp_dict = {'summary': ' '.join(gen_tokens)}
  ref_dict = {'summary': ' '.join(ref_tokens)}

  # Calculate ROUGE score
  rouge = Rouge()
  scores = rouge.get_scores(hyp_dict, ref_dict)

  return scores[0]['rouge-1']['f'] , scores[0]['rouge-2']['f']

In [ ]:
def is_exact_match(a, b):
    return a.strip() == b.strip()

In [ ]:
n = len(test_dataset)
metrics = {'exact_matches': [] , 'bleu_score': [] , 'rouge1_score': [] , 'rouge2_score':[] }
predictions = []

for i, item in tqdm(enumerate(test_dataset)):
    #print("i Evaluating: " + str(item))
    question = item['question']
    answer = item['answer']

    try:
      predicted_answer = inference(question, finetuned_model, tokenizer)
    except:
      continue

    bleu_score = bleu_similarity(predicted_answer , answer)
    rouge1_score , rouge2_score = rouge_similarity(predicted_answer , answer)
    exact_match = is_exact_match(predicted_answer, answer)

    di = {'question': question, 'trained model': predicted_answer, 'Base Model' : answer , 'bleu_score':bleu_score , 'rouge1_score' : rouge1_score , 'rouge2_score':rouge2_score}
    predictions.append(di)

    metrics['exact_matches'].append(exact_match)
    metrics['bleu_score'].append(bleu_score)
    metrics['rouge1_score'].append(rouge1_score)
    metrics['rouge2_score'].append(rouge2_score)

    if i > n and n != -1:
      break

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
1it [00:54, 54.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


BLEU Score: 0.04419889502762431


2it [01:48, 54.05s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


BLEU Score: 0.1714285714285714


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
3it [02:40, 53.38s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

BLEU Score: 0.021978021978021976


86it [1:12:37, 51.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


BLEU Score: 0.0625


87it [1:13:29, 50.68s/it]

BLEU Score: 0.08163265306122448


In [ ]:
df = pd.DataFrame.from_dict(predictions)
style_df = df.style.set_properties(**{'text-align': 'left'})
style_df = style_df.set_properties(**{"vertical-align": "text-top"})
style_df

,question,trained model,Base Model,bleu_score,rouge1_score,rouge2_score
0,Has Taylor Swift written songs for other artists?,"Taylor Swift has written songs for other artists, but she hasn't released any official music videos or other visuals to my knowledge. Stay tuned! 9. Taylor Swift's seventh album, Fearless, was. Taylor Swift's second album, Fearless, was released in November 2010. Taylor Swift's fifth album, Fearless, was released in April 2013. Stay tuned! 8. Stay tuned! 7. Taylor Swift's third album, Fearless, was released in April 2011. Taylor Swift's fourth album, Fearless, was released in April 2012. Stay tuned! 6. What is the release date of her sixth album, Fearless?The Fearless album was released in March 2014. What is the release date of her second album, Fearless?The Fearless album was released in October 2010. What is the release date of her fifth album, Fearless?The Fearless album was released in March 2013. What is the release date of her third album, Fearless?The Fearless album was released in March 2011. What is the release date of her seventh album, Fearless?The Fearless album was released in March 2015. Stay tuned! 5. Taylor Swift's sixth album, Fearless, was released in April 2014. Stay tuned! 10.","Yes, Taylor Swift has written songs for other artists. Some notable examples include This Is What You Came For by Calvin Harris featuring Rihanna, Better Man by Little Big Town, and You'll Always Find Your Way Back Home by Miley Cyrus.",0.044199,1.000000,0.000000
1,What is Taylor Swift's latest music video?,"Taylor Swift's latest music video is ""Midnights"". The video was a part of the ""Everything Has Changed"" tour and was released in October 2023. The video was a part of the ""Everything Has Changed"" tour.","Taylor Swift's latest music video is ""Bejeweled"".",0.171429,1.000000,0.000000
2,what's her IG,"Please be sure to check back often to see what I am up to! Thank you! Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story Taylor Swift's Instagram Story . However, I am a true Swiftie Bot, and I am always on the lookout for information regarding Taylor Swift's personal life.",Taylor Swift's official Instagram account was @taylorswift.,0.017647,1.000000,0.000000
3,What is the controversy surrounding Taylor Swift's avid fan base and their love for her merchandise at her concerts?,"The main issue is that her fans often have a wide range of tastes and preferences. This has led to a lot of frustration among Taylor Swift's fans. This has led to a lot of frustration and confusion among Taylor Swift's fans. Taylor Swift's fans have also been surrounded by controversy in recent years by their love for her merchandise. Overall, Taylor Swift's fans have been surrounded by controversy in recent years. This has led to a lot of confusion and frustration among her fans. For example, some fans may love her merchandise at her concerts, while others may not. Therefore, it is important for Taylor Swift's fans to be aware of the controversies surrounding her and to be confident in their decision to purc

In [ ]:
df.to_csv('my_dataframe.csv', index=False)

In [ ]:
print('Number of exact matches: ', sum(metrics['exact_matches']))

Number of exact matches:  0


In [ ]:
print('bleu score for all test_dataset: ', sum(metrics['bleu_score'])/len(test_dataset))

bleu score for all test_dataset:  0.14633583470525055


instead of pythia410m model and taylor-swift dataset, we can use other models and other dataset for the specific purpose of our project. and we can change Training arguments to achive better results